# AI Agents in .NET


## What is AI Agent?

AI agents are more than just AI models that generate text—they are autonomous systems that perceive, reason, and act. 

Unlike traditional AI, which simply responds based on static data, AI agents can interact with their environment, call tools, remember previous actions, and make decisions. 

## Setting up LLM

But before we begin we need to do something else.\
As we use LLM as the 'brain' of our agent, we need to set up it first. There are different ways how we can do it
- we can do it using [openAI platform](https://platform.openai.com/api-keys)
- we can use our own local model with [Ollama](https://ollama.com)
- we can use Azure OpenAI and [Azure AI Foundry](https://ai.azure.com) to deploy any LLM available for us

In this example we will be using AzureAI. 
>If you decide to go with another option you might require a bit different nuget packages

In [4]:
var Endpoint = "<REPLACE_WITH_AZURE_OPENAI_ENDPOINT>";
var Key = "<REPLACE_WITH_AZURE_OPENAI_API_KEY>";
var ChatModelId = "gpt-4o"; // use any model you deployed

## Semantic Kernel

In [10]:
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.41.0"

Installed Packages Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.41.0

In [3]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [4]:
var kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(ChatModelId, Endpoint, Key)
        .Build();

### Creating Plugin

With our Kernel created we need to give it more power.  
Plugin or a tool is basically an "instrument" that we are giving to our agent.  

Let's create a 'Weather Plugin' so our agent could discover what's the weather look like and use this information in case if it needs it.

In [5]:
using Microsoft.SemanticKernel;
using System.ComponentModel;

public class WeatherPlugin
{
    [KernelFunction("gets_the_weather")]
    [Description("Gets the weather")]
    public string GetWeather(string destination)
    {
        return $"It's {(Random.Shared.NextDouble() > 0.5 ? "sunny" : "raining")} in {destination}";
    }
    
    [KernelFunction("gets_the_wind_speed")]
    [Description("Gets the wind speed")]
    public string GetWindSpeed()
    {
        return $"{Random.Shared.Next(0, 50)} m/s";
    }
}

We created a plugin 'Weather Plugin' and implemented 2 methods to get weather and speed of wind information.  
Of course in a real application we would rather use real weather api but for the learning purposes that will work.

Now let's "give" our agent this tool. For this we need to register it

In [6]:
kernel.Plugins.AddFromType<WeatherPlugin>();

### Configuring Kernel

And we are almost done.

Now we can configure our kernel, set required service, configure our execution settings

In [7]:
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();
        
OpenAIPromptExecutionSettings openAiPromptExecutionSettings = new()
{
    Temperature = 0,
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

As we mentioned before, we don't usually enforce our agent to use tools, we are providing them to it and allow to use if them seem fit. Tha'ts why we set FunctionChoiceBehavior to Auto.

Now let's initiate our chat and check the reply

In [8]:
var chatHistory = new ChatHistory("How's it outside?");
        
await foreach (var message in chatCompletionService.GetStreamingChatMessageContentsAsync(
    chatHistory,
    executionSettings: openAiPromptExecutionSettings,
    kernel: kernel))
{
    Console.Write(message);
}

It's sunny outside, and the wind speed is 17 m/s.

We got our answer and as you can see neither we specified that the agent had to use out tool nor did we even mention the weather.
- perceive phase - LLM is intelligable enough to understand our question
- reason phase - analysing the question, the context, existing capabilities
- act phase - using our fake API to get information about the weather


# Microsoft.Extensions.AI

Semantic Kernel is an awesome framework and is great for a more complex scenario like creating multi agents collaboration. But for the simple agent like this is a bit overkill. 
Luckily for us Microsoft has recently introduced new package [Microsoft.Extension.AI](https://devblogs.microsoft.com/dotnet/introducing-microsoft-extensions-ai-preview/)
This package provides the .NET ecosystem with essential abstractions for integrating AI services into .NET applications and libraries, along with middleware for adding key capabilities.

Let's check how the same agent will look like

First let's install our packages

In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3"
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.4"

Installed Packages Azure.AI.OpenAI, 2.2.0-beta.4 Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3

In [2]:
using System.ClientModel;
using System.ComponentModel;
using Azure.AI.OpenAI;
using Microsoft.Extensions.AI;

In [5]:
var client = new AzureOpenAIClient(new Uri(Endpoint), new ApiKeyCredential(Key))
            .AsChatClient(ChatModelId)
            .AsBuilder()
            .UseFunctionInvocation() // allows to call registered functions
            .Build();

### Configuring Chat Client

As you can see the syntax is a bit different but more or less the same

Now let's configure it with our tools, but in this case there is no need for a separate plugin, we can just add these functions right here

In [6]:
[Description("Gets the weather")]
string GetWeather() => Random.Shared.NextDouble() > 0.5 ? "It's sunny" : "It's raining";

[Description("Gets the wind speed")]
string GetWindSpeed() => $"{Random.Shared.Next(0,50)} m/s";

Now we should register these tools in our agent

In [7]:
var chatOptions = new ChatOptions
{
    Tools = 
    [
        AIFunctionFactory.Create(GetWeather),
        AIFunctionFactory.Create(GetWindSpeed)
    ]
};

And that's really it.  
Let's now use it in a similar manner

In [8]:
await foreach (var message in client.GetStreamingResponseAsync("How's it outside?", chatOptions))
{
    Console.Write(message);
}

It's sunny outside, and the wind speed is 25 m/s.

### Final Thoughts

We learnt how to create AI agent in .NET with 2 different approaches:
- Semantic Kernel framework
- Microsoft.Extensions.AI package